In [16]:
import pandas as pd
import numpy as np
from osgeo import gdal, osr, ogr
import sys
import math
import os
import subprocess

In [17]:
SPATIAL_IDX = 5
TEMPORAL_IDX = 9

In [18]:
'''
Open input file <filename> and return numpy array
'''

def getInputData(filename):
    # This allows gdal to throw python exceptions
    gdal.UseExceptions()
    try:
        modisRaster = gdal.Open(filename)
    except e:
        print("Unable to open tif file")
        print(e)
        print(modisRaster.GetMetadata())
    modisData = modisRaster.ReadAsArray()
    # set NaN values to 0
    modisData = np.where((modisData < 1) | (modisData > 366), 0, modisData)
    modisData = np.floor(modisData)
    return modisData,modisRaster
    

In [19]:
'''
Set the number of spatial window based on row/col and SPATIAL_IDX
'''
def setSpatialWindow(row,col,arrayDims):
    top = max(0,row-SPATIAL_IDX)
    bottom = min(arrayDims[0],row+SPATIAL_IDX+1)
    left = max(0,col-SPATIAL_IDX)
    right = min(arrayDims[1],col+SPATIAL_IDX+1)
    return top,bottom,left,right
    

In [20]:
'''
Process the input array determining fire events based on SPATIAL_IDX and TEMPORAL_IDX
'''
def getEventData(inputArray):
    nextId = 1
    eventArray = np.zeros(inputArray.shape) #initialize event array to 0s
    #Traverse pixels in the MODIS Data Array to look for neighboring event pixels that belong to the event
    for row in range(inputArray.shape[0]):
        for col in range(inputArray.shape[1]):
            if inputArray[row,col] : # burn detected
                #print("{},{} burn detected".format(row,col))
                burnDay = inputArray[row,col]
                #print("Next ID={}, burnDay={}:".format(nextId,burnDay))
                # set spatial input and temporal bounds and create spatial windows
                top,bottom,left,right = setSpatialWindow(row,col,inputArray.shape)
                inputWindow = inputArray[top:bottom,left:right]
                temporalMask = np.logical_and(inputWindow>0,abs(inputWindow-burnDay)<=TEMPORAL_IDX)
                eventWindow = eventArray[top:bottom,left:right]
                
                currId = eventArray[row,col] # Event ID for current pixel
                overlapMask = np.logical_and(np.logical_and(eventWindow>0,temporalMask),eventWindow!=currId)
                if overlapMask.any():
                    #true overlap
                    overlapId = np.amin(eventWindow[overlapMask],axis=None)
                    eventWindow[temporalMask] = overlapId
                    if currId != 0:
                        #print("Event merge: setting {} to {}".format(currId,overlapId))
                        eventArray[eventArray==currId] = overlapId
                elif currId == 0: # no overlap and not set so get next ID
                    eventWindow[temporalMask] = nextId
                    nextId += 1
                else: 
                    eventWindow[temporalMask] = currId # propagate current id
    return eventArray

In [21]:
'''
Save the event data to GTiff file
'''
def saveEventData(eventData,input_raster,outfile):
    print("Saving to {}".format(outfile))
    rows = input_raster.RasterYSize
    cols = input_raster.RasterXSize
    output_raster = gdal.GetDriverByName('GTiff').Create(outfile, cols, rows, 1 ,gdal.GDT_Float32)
    output_raster.SetGeoTransform(input_raster.GetGeoTransform())
    output_raster.SetProjection(input_raster.GetProjection())
    output_raster.GetRasterBand(1).WriteArray(eventData) 

In [22]:
'''
Calculate event statistics for each fire event
'''
def getEventStats(eventData,inputData):
    print("Getting Event Stats...")
    eventid, pixels = np.unique(eventData, return_counts=True)
    df=pd.DataFrame({'eventID':eventid, 'pixels':pixels})
    df = df[df.eventID != 0] # drop '0
    df['km2'] = (df.pixels * 463*463)/1000000
    df['firstbd'] = df.eventID.apply(lambda x: inputData[eventData==x].min())
    df['lastbd'] = df.eventID.apply(lambda x: inputData[eventData==x].max())
    df['duration'] = df.lastbd - df.firstbd + 1
    df['fsr'] = df.pixels/df.duration
    df['maxbd'] = df.eventID.apply(lambda x: np.bincount(inputData[eventData==x].astype(int)).argmax())
    df_e_all = []
    print("Getting Daily Stats...")
    events = eventid[eventid>0] #exclude pixels set to 0 (no event)
    for e in events:
        bd,bd_pixels = np.unique(inputData[eventData==e],return_counts=True)
        df_e = pd.DataFrame({'eventID':e,'bd':bd,'pixels':bd_pixels})
        df_e['area_km2'] = (df_e.pixels * 463*463)/1000000
        df_e_all.append(df_e)
    df_detail = pd.concat(df_e_all)
    return df,df_detail


In [23]:
def createEventRaster(df,df_detail,modisData,eventData):
    print("Creating event raster")
    l_1 = np.zeros((eventData.shape))
    print(df.loc[df['eventID'] == 9.0])
    #last_bd = df.loc[df['eventID'] == 9.0,'lastbd']
    last_bd = df.loc[df['eventID'] == 9.0,'lastbd']
    print("Explicit")
    print(df.iloc[6]['lastbd'])
    print("Index")
    idx = df.index([df.eventID==9])
    print("idx={}".format(idx))
    print(df.iloc[idx]['lastbd'])
    km2 = lambda x: df.loc[df['eventID'] == x].km2
    #l_1(km2)
    print(l_1.shape)
    

In [24]:
def processFireEvent(infile,output_base,s,t):
    inputData,inputRaster = getInputData(infile)
    eventData = getEventData(inputData)
    saveEventData(eventData,inputRaster,(output_base + '.tif'))

In [25]:
def arraytoCSV(a,filename):
    df = pd.DataFrame(a)
    df.to_csv(filename)

In [ ]:
input_path = '../data/MCD64A1/C6/yearly_composites/'
output_path = '../data/MCD64A1/C6/yearly_composites_15x15/'

for s in range(1,16): # 1 to 15
    for t in range(1,16): # 1 to 15
        for year in range(2001,2018):
            if not os.path.exists(output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) + '.tif'):
                print("s={} t={} year={}".format(s,t,year))
                infile = input_path + 'USA_BurnDate_' + str(year) + '_ms.tif'
                output_base =  output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) 
                processFireEvent(infile,output_base,s,t)
                subprocess.call(['aws', 's3', 'sync', output_path, 
                                 's3://earthlab-natem/modis-burned-area/MCD64A1/C6/yearly_composites_15x15'])
                os.remove(output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) + '.tif')
            else: 
                print("deleting... s={} t={} year={}".format(s,t,year))
                os.remove(output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) + '.tif')
                

deleting... s=1 t=1 year=2001
deleting... s=1 t=1 year=2002
deleting... s=1 t=1 year=2003
deleting... s=1 t=1 year=2004
deleting... s=1 t=1 year=2005
deleting... s=1 t=1 year=2006
deleting... s=1 t=1 year=2007
deleting... s=1 t=1 year=2008
deleting... s=1 t=1 year=2009
deleting... s=1 t=1 year=2010
deleting... s=1 t=1 year=2011
deleting... s=1 t=1 year=2012
deleting... s=1 t=1 year=2013
deleting... s=1 t=1 year=2014
deleting... s=1 t=1 year=2015
deleting... s=1 t=1 year=2016
deleting... s=1 t=1 year=2017
deleting... s=1 t=2 year=2001
deleting... s=1 t=2 year=2002
deleting... s=1 t=2 year=2003
deleting... s=1 t=2 year=2004
deleting... s=1 t=2 year=2005
deleting... s=1 t=2 year=2006
deleting... s=1 t=2 year=2007
deleting... s=1 t=2 year=2008
deleting... s=1 t=2 year=2009
deleting... s=1 t=2 year=2010
deleting... s=1 t=2 year=2011
deleting... s=1 t=2 year=2012
deleting... s=1 t=2 year=2013
deleting... s=1 t=2 year=2014
deleting... s=1 t=2 year=2015
deleting... s=1 t=2 year=2016
deleting..

deleting... s=2 t=3 year=2010
deleting... s=2 t=3 year=2011
deleting... s=2 t=3 year=2012
deleting... s=2 t=3 year=2013
deleting... s=2 t=3 year=2014
deleting... s=2 t=3 year=2015
deleting... s=2 t=3 year=2016
deleting... s=2 t=3 year=2017
deleting... s=2 t=4 year=2001
deleting... s=2 t=4 year=2002
deleting... s=2 t=4 year=2003
deleting... s=2 t=4 year=2004
deleting... s=2 t=4 year=2005
deleting... s=2 t=4 year=2006
deleting... s=2 t=4 year=2007
deleting... s=2 t=4 year=2008
deleting... s=2 t=4 year=2009
deleting... s=2 t=4 year=2010
deleting... s=2 t=4 year=2011
deleting... s=2 t=4 year=2012
deleting... s=2 t=4 year=2013
deleting... s=2 t=4 year=2014
deleting... s=2 t=4 year=2015
deleting... s=2 t=4 year=2016
deleting... s=2 t=4 year=2017
deleting... s=2 t=5 year=2001
deleting... s=2 t=5 year=2002
deleting... s=2 t=5 year=2003
deleting... s=2 t=5 year=2004
deleting... s=2 t=5 year=2005
deleting... s=2 t=5 year=2006
deleting... s=2 t=5 year=2007
deleting... s=2 t=5 year=2008
deleting..

deleting... s=3 t=9 year=2010
deleting... s=3 t=9 year=2011
deleting... s=3 t=9 year=2012
deleting... s=3 t=9 year=2013
deleting... s=3 t=9 year=2014
deleting... s=3 t=9 year=2015
deleting... s=3 t=9 year=2016
deleting... s=3 t=9 year=2017
deleting... s=3 t=10 year=2001
deleting... s=3 t=10 year=2002
deleting... s=3 t=10 year=2003
deleting... s=3 t=10 year=2004
deleting... s=3 t=10 year=2005
deleting... s=3 t=10 year=2006
deleting... s=3 t=10 year=2007
deleting... s=3 t=10 year=2008
deleting... s=3 t=10 year=2009
deleting... s=3 t=10 year=2010
deleting... s=3 t=10 year=2011
deleting... s=3 t=10 year=2012
deleting... s=3 t=10 year=2013
deleting... s=3 t=10 year=2014
deleting... s=3 t=10 year=2015
deleting... s=3 t=10 year=2016
deleting... s=3 t=10 year=2017
deleting... s=3 t=11 year=2001
deleting... s=3 t=11 year=2002
deleting... s=3 t=11 year=2003
deleting... s=3 t=11 year=2004
deleting... s=3 t=11 year=2005
deleting... s=3 t=11 year=2006
deleting... s=3 t=11 year=2007
deleting... s=3 

deleting... s=5 t=2 year=2002
deleting... s=5 t=2 year=2003
deleting... s=5 t=2 year=2004
deleting... s=5 t=2 year=2005
deleting... s=5 t=2 year=2006
deleting... s=5 t=2 year=2007
deleting... s=5 t=2 year=2008
deleting... s=5 t=2 year=2009
deleting... s=5 t=2 year=2010
deleting... s=5 t=2 year=2011
deleting... s=5 t=2 year=2012
deleting... s=5 t=2 year=2013
deleting... s=5 t=2 year=2014
deleting... s=5 t=2 year=2015
deleting... s=5 t=2 year=2016
deleting... s=5 t=2 year=2017
deleting... s=5 t=3 year=2001
deleting... s=5 t=3 year=2002
deleting... s=5 t=3 year=2003
deleting... s=5 t=3 year=2004
deleting... s=5 t=3 year=2005
deleting... s=5 t=3 year=2006
deleting... s=5 t=3 year=2007
deleting... s=5 t=3 year=2008
deleting... s=5 t=3 year=2009
deleting... s=5 t=3 year=2010
deleting... s=5 t=3 year=2011
deleting... s=5 t=3 year=2012
deleting... s=5 t=3 year=2013
deleting... s=5 t=3 year=2014
deleting... s=5 t=3 year=2015
deleting... s=5 t=3 year=2016
deleting... s=5 t=3 year=2017
deleting..

deleting... s=6 t=5 year=2001
deleting... s=6 t=5 year=2002
deleting... s=6 t=5 year=2003
deleting... s=6 t=5 year=2004
deleting... s=6 t=5 year=2005
deleting... s=6 t=5 year=2006
deleting... s=6 t=5 year=2007
deleting... s=6 t=5 year=2008
deleting... s=6 t=5 year=2009
deleting... s=6 t=5 year=2010
deleting... s=6 t=5 year=2011
deleting... s=6 t=5 year=2012
deleting... s=6 t=5 year=2013
deleting... s=6 t=5 year=2014
deleting... s=6 t=5 year=2015
deleting... s=6 t=5 year=2016
deleting... s=6 t=5 year=2017
deleting... s=6 t=6 year=2001
deleting... s=6 t=6 year=2002
deleting... s=6 t=6 year=2003
deleting... s=6 t=6 year=2004
deleting... s=6 t=6 year=2005
deleting... s=6 t=6 year=2006
deleting... s=6 t=6 year=2007
deleting... s=6 t=6 year=2008
deleting... s=6 t=6 year=2009
deleting... s=6 t=6 year=2010
deleting... s=6 t=6 year=2011
deleting... s=6 t=6 year=2012
deleting... s=6 t=6 year=2013
deleting... s=6 t=6 year=2014
deleting... s=6 t=6 year=2015
deleting... s=6 t=6 year=2016
deleting..

...

s=9 t=9 year=2002
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2002s9t9.tif
s=9 t=9 year=2003
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2003s9t9.tif
s=9 t=9 year=2004
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2004s9t9.tif
s=9 t=9 year=2005
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2005s9t9.tif
s=9 t=9 year=2006
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2006s9t9.tif
s=9 t=9 year=2007
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2007s9t9.tif
s=9 t=9 year=2008
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2008s9t9.tif
s=9 t=9 year=2009
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2009s9t9.tif
s=9 t=9 year=2010
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2010s9t9.tif
s=9 t=9 year=2011
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2011s9t9.tif
s=9 t=9 year=2012
Saving to ..

Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2017s9t13.tif
s=9 t=14 year=2001
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2001s9t14.tif
s=9 t=14 year=2002
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2002s9t14.tif
s=9 t=14 year=2003
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2003s9t14.tif
s=9 t=14 year=2004
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2004s9t14.tif
s=9 t=14 year=2005
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2005s9t14.tif
s=9 t=14 year=2006
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2006s9t14.tif
s=9 t=14 year=2007
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2007s9t14.tif
s=9 t=14 year=2008
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2008s9t14.tif
s=9 t=14 year=2009
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2009s9t14.tif
s=9 t=14 year=2010
Saving to 

s=10 t=3 year=2015
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2015s10t3.tif
s=10 t=3 year=2016
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2016s10t3.tif
s=10 t=3 year=2017
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2017s10t3.tif
s=10 t=4 year=2001
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2001s10t4.tif
s=10 t=4 year=2002
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2002s10t4.tif
s=10 t=4 year=2003
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2003s10t4.tif
s=10 t=4 year=2004
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2004s10t4.tif
s=10 t=4 year=2005
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2005s10t4.tif
s=10 t=4 year=2006
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2006s10t4.tif
s=10 t=4 year=2007
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2007s10t4.tif
s=10 t=4 y

s=10 t=8 year=2013
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2013s10t8.tif
s=10 t=8 year=2014
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2014s10t8.tif
s=10 t=8 year=2015
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2015s10t8.tif
s=10 t=8 year=2016
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2016s10t8.tif
s=10 t=8 year=2017
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2017s10t8.tif
s=10 t=9 year=2001
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2001s10t9.tif
s=10 t=9 year=2002
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2002s10t9.tif
s=10 t=9 year=2003
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2003s10t9.tif
s=10 t=9 year=2004
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2004s10t9.tif
s=10 t=9 year=2005
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2005s10t9.tif
s=10 t=9 y

s=10 t=13 year=2010
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2010s10t13.tif
s=10 t=13 year=2011
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2011s10t13.tif
s=10 t=13 year=2012
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2012s10t13.tif
s=10 t=13 year=2013
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2013s10t13.tif
s=10 t=13 year=2014
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2014s10t13.tif
s=10 t=13 year=2015
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2015s10t13.tif
s=10 t=13 year=2016
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2016s10t13.tif
s=10 t=13 year=2017
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2017s10t13.tif
s=10 t=14 year=2001
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2001s10t14.tif
s=10 t=14 year=2002
Saving to ../data/MCD64A1/C6/yearly_composites_15x15/USA_BurnDate_2002s